In [ ]:
# default_exp model
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Model

> In this notebook you create and test a Python class to hold your machine learning model.

***input***: toy dataset from data-notebook

***output***: python module containing ML model class 

***description:***

In this notebook you hypothetize, explain and explore machine learning models to solve your problem.

Remember to add `# export` to top of all cells containing functions or classes that you have defined and want to use outside this notebook.

Edit this and the other cells to describe your project.

## Import relevant modules

In [ ]:
# export
import numpy as np

# your code here

## Define notebook parameters

Remember, only simple assignments here!

In [ ]:
# this cell is tagged with 'parameters'
seed = 0

Make immediate derivations from the parameters:

In [ ]:
np.random.seed(seed)

## Import toy data for testing

In [ ]:
# your code here

> Note, that depending on your choice of file format and your variables, you might have to redefine data types once you load data! 

## Explain the math behind

## Explore with simple scripts before constructing the model class

In [ ]:
# your code here

## Define you ML model class

Remember that you can also create base class and subclasses to utilize heritance!

In [ ]:
# export

# your code here

## Unit test

Unit test your class or classes

In [ ]:
# your code here

## Visualize model behaviour with toy data

In [ ]:
## your code here

## Output of this notebook

The result of this notebook is a collection methods ready for evaluation with the real data.

You should export classes and functions to `model.py` with `# nbdev_build_lib` (workflows will do this automatically).

## You can move on to loss notebook!